# Basic scene generation and visualization

 ## Setup code

In [ ]:
# setup vulkan
!mkdir -p /usr/share/vulkan/icd.d
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json
!mv nvidia_icd.json /usr/share/vulkan/icd.d
!mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
!apt-get install -y --no-install-recommends libvulkan-dev
# dependencies
!pip install --upgrade mani_skill tyro

In [ ]:
!pip install torch torchvision
!pip install scene-synthesizer[recommend]
!pip install usd-core
!pip install "pyglet<2"
!pip install hydra-core

Clone private repo. You can specify your password or generate one-time token.

In [ ]:
# SECURE?
import os
from getpass import getpass
user = getpass('GitHub user')
pswd_or_token = getpass('GitHub password / token')
os.environ['GITHUB_AUTH'] = user + ':' + pswd_or_token
!git clone https://$GITHUB_AUTH@gitlab.2a2i.org/cv/robo/darkstore-synthesizer

Next download zip with assets from Google Drive and Robocassa assets.

In [ ]:
!gdown 1iiclwZDisbP2afBxG79rBtbRiewHFt7W
!unzip assets.zip
!python -m mani_skill.utils.download_asset RoboCasa

## Scene generation

In [ ]:
import hydra
from omegaconf import OmegaConf
from hydra.core.config_store import ConfigStore
import os
from pathlib import Path
import json
import sys

sys.path.append('darkstore-synthesizer')
from dsynth.scene_gen.arrangements import shelf_placement_v2
from dsynth.scene_gen.layouts.random_connectivity import add_many_zones, get_orientation
from dsynth.scene_gen.utils import flatten_dict
from dsynth.scene_gen.hydra_configs import product_filling_from_darkstore_config, Config, ShelfConfig
from dsynth.assets.asset import load_assets_lib

In [ ]:
# Specify output directory
from pathlib import Path
output_dir = 'generated_scenes'
output_dir = Path(output_dir)
output_dir.mkdir()

In [ ]:
# Instantiate base configs
cs = ConfigStore.instance()
cs.store(group="shelves", name="base_shelf_config", node=ShelfConfig)
cs.store(group="ds", name="main_darkstore_config_base", node=Config)

In [ ]:
!cat darkstore-synthesizer/conf/ds/config.yaml

In [ ]:
!cat darkstore-synthesizer/conf/ds/zones_list.yaml

In [ ]:
!cat darkstore-synthesizer/conf/zones/milk_zone.yaml
!cat darkstore-synthesizer/conf/zones/grocery.yaml

In [ ]:
!cat darkstore-synthesizer/conf/shelves/full_milk.yaml
!echo ============
!cat darkstore-synthesizer/conf/shelves/grocery.yaml
!echo ============
!cat darkstore-synthesizer/conf/shelves/drinks.yaml

In [ ]:
# Initialze scene config
overrides = [
   "~ds.zones.zone2",
   "ds.size_n=1",
  "ds.size_m=2"
    ]
with hydra.initialize(version_base=None, config_path="darkstore-synthesizer/conf"):
    cfg = hydra.compose(overrides=overrides, config_name='config')
    print(cfg)

In [ ]:
cfg.ds

In [ ]:
product_assets_lib = flatten_dict(load_assets_lib(cfg.assets.products_hierarchy), sep='.')

In [ ]:
list(product_assets_lib.keys())

In [ ]:
product_filling = product_filling_from_darkstore_config(cfg.ds, list(product_assets_lib.keys()))
zones_dict = {key: list(val.keys()) for key, val in product_filling.items()}
product_filling_flattened = flatten_dict(product_filling, sep='.')

In [ ]:
zones_dict

In [ ]:
n, m = cfg.ds.size_n, cfg.ds.size_m
x, y = cfg.ds.entrance_coords_x, cfg.ds.entrance_coords_y
mat = [[0] * m for _ in range(n)]
is_gen, room = add_many_zones((x, y), mat, zones_dict)
assert is_gen, "Try again"
is_rotate = get_orientation((x, y), room)

In [ ]:
room

In [ ]:
scene_meta = shelf_placement_v2(product_filling_flattened, room, is_rotate, product_assets_lib, cfg.ds.show)

In [ ]:
with open(output_dir / "scene_config.json", "w") as f:
    json.dump(scene_meta, f, indent=4)

with open(output_dir / "input_config.yaml", "w") as f:
    f.write(OmegaConf.to_yaml(cfg))

## Open generated scene in ManiSkill simulator

In [ ]:
import time
from tqdm import tqdm
import gymnasium as gym

from IPython.display import Video

import torch
import mani_skill.envs
from mani_skill.utils.wrappers import RecordEpisode

from dsynth.envs.pick_to_cart import PickToCartEnv

In [ ]:
# Specify shader used for rendering
shader = 'default' # enable raytracing (not working in colab?)

# Style ID for floor and walls from Robocasa assets
style_id = 0

# Create environment
env = gym.make('PickToCartEnv',
                   robot_uids='fetch',
                   config_dir_path = output_dir,
                   style_ids = [style_id],
                   num_envs=1,
                   viewer_camera_configs={'shader_pack': shader},
                   human_render_camera_configs={'shader_pack': shader},
                   render_mode="rgb_array",
                   enable_shadow=True,
              #  obs_mode='rgbd',
                   )

In [ ]:
# Save rollout video and trajectory
new_traj_name = time.strftime("%Y%m%d_%H%M%S")
video_path = output_dir / f"./videos_style={style_id}_shader={shader}"
env = RecordEpisode(
    env,
    output_dir=video_path,
    trajectory_name=new_traj_name,
    save_video=True,
    video_fps=30,
    avoid_overwriting_video=True
)

In [ ]:
# Run a rollout
episode_length = 20
env.reset()

zero_action = torch.zeros_like(torch.from_numpy(env.action_space.sample()))

for i in tqdm(range(episode_length)):
    obs, reward, terminated, truncated, info = env.step(zero_action)

env.close()

In [ ]:
Video(f"generated_scenes/videos_style={style_id}_shader={shader}/1.mp4", embed=True, width=640) # Watch our replay